<div dir="rtl">

# רשימות מקושרות (Linked Lists) ב‑C# — מחברת לימוד

במחברת הזאת נלמד את הרעיון של **<span style="color: #e91e63">רשימה מקושרת</span>  חד־כיוונית ודו כיוונית**  (Double Linked List) (Singly Linked List), נבנה אותה ב‑C# ונראה איך לבצע פעולות נפוצות. 

---

## 📑 תוכן עניינים

| חלק | נושא |
|:---:|:-----|
| **א'** | מבוא — מהי רשימה מקושרת, איך עובדת, למה צריך |
| **ב'** | שימושים נפוצים — Stack, Queue, LRU, HashMap |
| **ג'** | API ומימוש — ממשק, טיפוסים, פעולות |
| **ד'** | קוד — Node, פונקציות עזר, היפוך רשימה |
| **ה'** | אנימציה ותרגילים |

---

### מה תלמדו כאן
- מהי <span style="color: #e91e63">רשימה מקושרת</span> ומהו "צומת" (Node)
- איך היא עובדת (הפניות / references בין צמתים)
- למה בכלל צריך את זה (לעומת מערך / List)
- בנייה, מעבר (Traversal), הוספה ומחיקה
- היפוך רשימה **במקום**: זמן ‎O(n)‎ וזיכרון נוסף ‎O(1)‎
- הדמיה (animation) בסיסית שמראה איך המצביעים זזים בכל צעד

</div>

<div dir="rtl">

### 1) מהי <span style="color: #e91e63">רשימה מקושרת</span>?

**<span style="color: #e91e63">רשימה מקושרת</span>** היא מבנה נתונים שמורכב מצמתים (Nodes).  
כל צומת מכיל:
- **ערך** (Value) — הנתון שאנחנו רוצים לשמור
- **הפניה לצומת הבא** (Next) — או `null` אם זה הצומת האחרון

אפשר לחשוב על זה כמו "שרשרת":
```
[1] -> [2] -> [3] -> null
```

בניגוד למערך, האיברים **לא חייבים להיות רציפים בזיכרון**. כל צומת יודע רק "מי הבא אחריי".

</div>

<div dir="rtl" style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 12px; margin: 30px 0;">
  <h2 style="margin: 0;">📘 חלק א': מבוא לרשימות מקושרות</h2>
  <p style="margin: 10px 0 0 0; opacity: 0.9;">מהי רשימה מקושרת, איך היא עובדת, ולמה בכלל צריך אותה</p>
</div>

<div dir="rtl">

### 2) איך זה עובד בפועל? (references)

ב‑C# הצומת הבא נשמר כ‑**reference** (הפניה לאובייקט).  
כשאנחנו משנים את `Next`, אנחנו בעצם "מחברים" מחדש את השרשרת.

לכן <span style="color: #e91e63">רשימות מקושרות</span> הן תרגול מעולה ל:
- חשיבה על **מצביעים/הפניות** (למרות שאין מצביעים גולמיים כמו ב‑C)
- עבודה נכונה עם `null`
- מניעת "איבוד" חלק מהרשימה (שמירת `next` לפני שינוי `Next`)

</div>

<div dir="rtl">

### 3) למה צריך <span style="color: #e91e63">רשימה מקושרת</span>? (ולמה לא תמיד)

#### יתרונות
- **הוספה/מחיקה בתחילת הרשימה**: ‎O(1)‎ (רק משנים הפניות)
- קל "לתפור" רשימות: לחבר/לחתוך קטעים ע"י שינוי `Next`
- מבנה בסיסי להרבה מבנים אחרים: תורים, מחסניות, Hash buckets ועוד

#### חסרונות
- **גישה לאיבר ה‑k**: ‎O(k)‎ (צריך לעבור אחד‑אחד)
- פחות ידידותי למטמון (CPU cache) לעומת מערך רציף
- יותר אובייקטים → יותר overhead בזיכרון וב‑GC
- פחות מתאים לפיתוח מכוון נתונים (data-oriented design)

#### מתי משתמשים?
- כשעיקר העבודה היא **הוספות/מחיקות** לאורך המבנה (בעיקר בתחילה/אמצע),  
  ולא גישה אקראית מהירה כמו במערך.

</div>

<div dir="rtl">

### שימושים נפוצים ל<span style="color: #e91e63">רשימה מקושרת</span> (Linked List)

<span style="color: #e91e63">רשימה מקושרת</span> היא מבנה נתונים פשוט אבל שימושי כשצריך **"לתפור"** אובייקטים אחד לשני בעזרת הפניות (`Next`), ולעשות **הוספות/מחיקות** בלי להזיז הרבה נתונים. בנוסף, היא משמשת כבסיס למבנים אחרים.

#### 1) Stack / Queue (מימושים קלאסיים)
- **Stack (מחסנית)**: `Push`/`Pop` בראש הרשימה — מהיר (**O(1)**).
- **Queue (תור)**: `Enqueue` בזנב ו-`Dequeue` בראש — מהיר (**O(1)**) אם שומרים גם `tail`.

#### 2) HashMap — טיפול בהתנגשויות (Chaining)
בהרבה מימושים של טבלאות גיבוב, כל *bucket* יכול להיות רשימה (לעיתים <span style="color: #e91e63">מקושרת</span>) של זוגות `(key, value)` כדי לטפל בהתנגשויות.  
זה מאפשר הוספה/מחיקה בתוך bucket בצורה פשוטה.

#### 3) LRU Cache — <span style="color: #e91e63">רשימה</span> דו־כיוונית + Dictionary
מימוש LRU קלאסי משתמש ב-**<span style="color: #e91e63">רשימה</span> דו־כיוונית** כדי להזיז פריטים לראש במהירות, וב-**Dictionary** כדי למצוא פריט ב-**O(1)**.
- גישה/עדכון פריט → העברה לראש
- חריגה מהקיבולת → הסרה מהזנב

#### 4) Free-List / Object Pool (ניהול "פנויים")
כשמנהלים אובייקטים/בלוקים פנויים (למשל במאגרים / pools), אפשר לקשר את הפנויים כ<span style="color: #e91e63">רשימה</span>:
- הקצאה/החזרה מהירה
- פחות הקצאות חדשות → פחות עומס על GC

#### 5) Undo/Redo & היסטוריה
היסטוריה של פעולות יכולה להיות רצף של מצבים/פקודות. עם **<span style="color: #e91e63">רשימה</span> דו־כיוונית** נוח לזוז קדימה/אחורה.  
(בפועל לפעמים משתמשים גם ב-Deque/מערך דינמי — תלוי בדפוס השימוש.)

---

### כלל אצבע
- בחר/י **<span style="color: #e91e63">רשימה מקושרת</span>** כשעיקר העבודה היא **הוספות/מחיקות** (במיוחד בראש/באמצע) ולא צריך אינדקס מהיר.
- בחר/י **Array/List** כשצריך **גישה אקראית מהירה** וביצועי cache טובים.

</div>

<div dir="rtl" style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); color: white; padding: 20px; border-radius: 12px; margin: 30px 0;">
  <h2 style="margin: 0;">🔧 חלק ב': שימושים נפוצים</h2>
  <p style="margin: 10px 0 0 0; opacity: 0.9;">Stack, Queue, HashMap, LRU Cache, ועוד</p>
</div>

<div dir="rtl">

### ממשק משתמש (API) ל<span style="color: #e91e63">רשימה מקושרת</span> — מה "רואים מבחוץ" וכיצד זה עובד

כשאומרים "ממשק משתמש" ל<span style="color: #e91e63">רשימה מקושרת</span>, הכוונה היא ל־**API**: אילו פעולות המשתמש/ת של המחלקה יכול/ה לבצע, מה הן מחזירות, ומה המשמעות והסיבוכיות שלהן.

להלן API טיפוסי ל<span style="color: #e91e63">רשימה מקושרת</span> חד־כיוונית (Singly Linked List), עם הסברים.

---

### 1) טיפוסים בסיסיים

#### `Node<T>` (צומת)
צומת הוא יחידת הבניין:
- `Value` — הערך
- `Next` — הפניה לצומת הבא

בד״כ `Node<T>` הוא **פרטי/פנימי** (internal/private) כדי שמי שמשתמש ב<span style="color: #e91e63">רשימה</span> לא "ישבור" את המצביעים מבחוץ.

#### `LinkedList<T>` (הרשימה)
המחלקה שמנהלת את השרשרת:
- מצביע לראש `Head`
- לפעמים גם מצביע לזנב `Tail` כדי לאפשר הוספה מהירה לסוף
- מונה `Count`

---

### 2) תכונות (Properties)

#### `int Count { get; }`
כמה איברים יש ב<span style="color: #e91e63">רשימה</span>.
- שימוש: לדעת גודל בלי לספור כל פעם.
- סיבוכיות: **O(1)** (אם שומרים מונה).

#### `bool IsEmpty { get; }`
נוחות: `Count == 0`.
- סיבוכיות: **O(1)**.

#### `T? First { get; }` / `T? Last { get; }` (אופציונלי)
הערך של הראש/הזנב.
- `First`: **O(1)** תמיד (אם יש `Head`)
- `Last`: **O(1)** רק אם שומרים `Tail`, אחרת **O(n)**

---

### 3) פעולות בסיסיות

#### `void AddFirst(T value)`
מוסיף איבר לראש ה<span style="color: #e91e63">רשימה</span>.
- מה קורה בפנים: `newNode.Next = Head; Head = newNode;`
- סיבוכיות: **O(1)**

#### `void AddLast(T value)`
מוסיף לסוף ה<span style="color: #e91e63">רשימה</span>.
- אם יש `Tail`: **O(1)**
- בלי `Tail`: צריך לעבור עד הסוף → **O(n)**

#### `T RemoveFirst()`
מסיר ומחזיר את הראש.
- מה קורה בפנים: `Head = Head.Next`
- סיבוכיות: **O(1)**
- הערה: אם ה<span style="color: #e91e63">רשימה</span> ריקה — לזרוק חריגה או להחזיר `bool`/`Try...`.

#### `bool Remove(T value)`
מסיר את *המופע הראשון* של ערך.
- צריך לעבור צומת-צומת עד שמוצאים → **O(n)**
- דורש שמירה על `prev` כדי "לתפור" את `prev.Next = curr.Next`.

#### `bool Contains(T value)`
בודק אם קיים ערך ב<span style="color: #e91e63">רשימה</span>.
- סיבוכיות: **O(n)**

---

### 4) מעבר על ה<span style="color: #e91e63">רשימה</span> (Traversal / Iteration)

#### `IEnumerable<T> AsEnumerable()` או `GetEnumerator()`
מאפשר `foreach`.
- סיבוכיות לכל מעבר מלא: **O(n)**
- למה חשוב: זו הדרך "הסטנדרטית" להשתמש ב<span style="color: #e91e63">רשימה</span> בלי לחשוף Nodes.

---

### 5) פעולות אלגוריתמיות נפוצות

#### `void Reverse()`
היפוך ה<span style="color: #e91e63">רשימה</span> במקום (In-place).
- סיבוכיות: **O(n)** זמן, **O(1)** זיכרון נוסף.

#### `T? FindMiddle()`
אמצע ה<span style="color: #e91e63">רשימה</span> בעזרת `slow/fast`.
- סיבוכיות: **O(n)**

#### `bool HasCycle()`
זיהוי מעגל (Floyd).
- סיבוכיות: **O(n)**

---

### 6) החלטות עיצוב חשובות (Design Notes)

#### האם לחשוף `Node<T>` החוצה?
בדרך כלל **לא**:
- אם המשתמש מחזיק Node ומשנה `Next`, הוא יכול ליצור באגים/מעגלים/ניתוקים.
- עדיף לחשוף ערכים ופעולות, לא מצביעים.

#### מה קורה כשמבקשים פעולה לא חוקית?
שתי גישות נפוצות:
1. חריגות: `InvalidOperationException` על `RemoveFirst` כשה<span style="color: #e91e63">רשימה</span> ריקה  
2. API "בטוח": `bool TryRemoveFirst(out T value)`

#### האם לשמור `Tail`?
- אם יש הרבה `AddLast`, שווה לשמור `Tail` כדי לקבל **O(1)**.
- אם כמעט תמיד עובדים בראש — אפשר לוותר.

#### האם לשמור `Count`?
כמעט תמיד כן — אחרת `Count` יהפוך ל־**O(n)** (ספירה כל פעם).

---

### 7) דוגמה קצרה ל־API "נקי" (רק חתימה)

להמחשה בלבד, בלי מימוש כאן:

- `int Count { get; }`
- `void AddFirst(T value)`
- `void AddLast(T value)`
- `T RemoveFirst()`
- `bool Remove(T value)`
- `bool Contains(T value)`
- `void Reverse()`
- `IEnumerable<T> AsEnumerable()` (או `GetEnumerator()`)

---

</div>

<div dir="rtl" style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); color: white; padding: 20px; border-radius: 12px; margin: 30px 0;">
  <h2 style="margin: 0;">📋 חלק ג': API וממשק</h2>
  <p style="margin: 10px 0 0 0; opacity: 0.9;">טיפוסים, תכונות, פעולות בסיסיות והחלטות עיצוב</p>
</div>

<div dir="rtl">

### 4) איך מריצים C# בתוך Jupyter?

כן — אפשר מחברת Jupyter עם **תאים של C#** בעזרת **.NET Interactive / Polyglot Notebooks**.

### אפשרות A: VS Code (לרוב הכי פשוט)
התקינו את ההרחבה **Polyglot Notebooks** ופתחו את קובץ ה‑`.ipynb` עם קרנל `.NET (C#)`. citeturn5search3

### אפשרות B: Jupyter / JupyterLab
הריצו:
```bash
dotnet tool install -g Microsoft.dotnet-interactive
dotnet interactive jupyter install
jupyter kernelspec list
```
ואז בחרו קרנל `.NET (C#)` / `.net-csharp`. citeturn10view0

</div>


<div dir="rtl" style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); color: white; padding: 20px; border-radius: 12px; margin: 30px 0;">
  <h2 style="margin: 0;">💻 חלק ד': מימוש בקוד</h2>
  <p style="margin: 10px 0 0 0; opacity: 0.9;">Node, פונקציות עזר, היפוך רשימה, הוספה ומחיקה</p>
</div>

In [ ]:
# Quick sanity check: if this prints, your C# kernel is working.
Console.WriteLine("✅ C# kernel is working!");

<div dir="rtl">

## 5) טיפוס הצומת Node<T> (גנרי)

<span style="color: #e91e63">רשימה מקושרת</span> חד־כיוונית מורכבת מצמתים.

בכל צומת נשמור:
- `Value` — הערך
- `Next` — הפניה לצומת הבא (או `null`)

נשתמש בצומת **גנרי** (`T`) כדי שאפשר יהיה לשמור כל טיפוס נתונים (int, string, אובייקטים וכו').

</div>

In [ ]:
#nullable enable
using System;
using System.Collections.Generic;
using System.Linq;

public sealed class Node<T>
{
    public T Value { get; set; }
    public Node<T>? Next { get; set; }

    public Node(T value, Node<T>? next = null)
    {
        Value = value;
        Next = next;
    }

    public override string ToString() => Value?.ToString() ?? "null";
}

<div dir="rtl">

## 6) פונקציות עזר

נבנה כמה כלים כדי שיהיה קל לעבוד:
- בניית <span style="color: #e91e63">רשימה</span> מתוך אוסף ערכים
- הפיכת ה<span style="color: #e91e63">רשימה</span> חזרה ל‑`IEnumerable<T>`
- הדפסה יפה בפורמט `a -> b -> c -> null`

</div>

In [ ]:
using System.Runtime.CompilerServices;
using System.Text;

static Node<T>? BuildList<T>(IEnumerable<T> values)
{
    Node<T>? head = null;
    Node<T>? tail = null;

    foreach (var v in values)
    {
        var n = new Node<T>(v);
        if (head is null)
        {
            head = n;
            tail = n;
        }
        else
        {
            tail!.Next = n;
            tail = n;
        }
    }
    return head;
}

static IEnumerable<T> ToEnumerable<T>(Node<T>? head)
{
    var cur = head;
    while (cur is not null)
    {
        yield return cur.Value;
        cur = cur.Next;
    }
}

static string ListToArrowString<T>(Node<T>? head, int maxNodes = 50)
{
    // Safety: prevent infinite loops if someone creates a cycle.
    var sb = new StringBuilder();
    var seen = new HashSet<int>();

    var cur = head;
    int count = 0;
    while (cur is not null && count < maxNodes)
    {
        int id = RuntimeHelpers.GetHashCode(cur);
        if (!seen.Add(id))
        {
            sb.Append(cur.Value);
            sb.Append(" -> (cycle)");
            return sb.ToString();
        }

        sb.Append(cur.Value);
        sb.Append(" -> ");
        cur = cur.Next;
        count++;
    }

    sb.Append(cur is null ? "null" : "...");
    return sb.ToString();
}

static void PrintList<T>(Node<T>? head, string? label = null)
{
    if (!string.IsNullOrWhiteSpace(label))
        Console.WriteLine(label);

    Console.WriteLine(ListToArrowString(head));
}

In [ ]:
var head = BuildList(new[] { 1, 2, 3, 4, 5 });
PrintList(head, "Original:");
Console.WriteLine($"As array: [{string.Join(", ", ToEnumerable(head))}]");

<div dir="rtl">

## 7) היפוך <span style="color: #e91e63">רשימה</span> חד־כיוונית (איטרטיבי, במקום)

**מטרה:** להפוך את כיוון ההפניות `Next`.

נחזיק שלושה משתנים:
- `prev` — החלק שכבר הפכנו
- `curr` — הצומת שאנחנו עובדים עליו כרגע
- `next` — שמירה זמנית כדי לא "לאבד" את המשך ה<span style="color: #e91e63">רשימה</span>

פסאודו‑קוד:
```
prev = null
curr = head
while curr != null:
    next = curr.Next
    curr.Next = prev
    prev = curr
    curr = next
return prev
```

סיבוכיות:
- זמן: ‎O(n)‎
- זיכרון נוסף: ‎O(1)‎

</div>

In [ ]:
static Node<T>? ReverseInPlace<T>(Node<T>? head)
{
    Node<T>? prev = null;
    var curr = head;

    while (curr is not null)
    {
        var next = curr.Next; // save
        curr.Next = prev;     // reverse pointer
        prev = curr;          // advance prev
        curr = next;          // advance curr
    }

    return prev; // new head
}

In [ ]:
var h1 = BuildList(new[] { 10, 20, 30, 40 });
PrintList(h1, "Before reverse:");
var reversed = ReverseInPlace(h1);
PrintList(reversed, "After reverse:");

<div dir="rtl">

### 8) “אנימציה”: לראות איך `prev` ו‑`curr` זזים

במחברת אפשר “להדגים” ע"י הצגת מצבי ביניים בכל צעד של האלגוריתם.

הקוד למטה מנסה לבצע עדכון חי של הפלט (Live Update).  
אם זה לא נתמך בסביבה שלכם, הוא עובר אוטומטית למצב שמדפיס “פריימים” אחד אחרי השני.

</div>


In [ ]:
using System.Threading.Tasks;
using Microsoft.DotNet.Interactive.Formatting;

// Renders the current state of the algorithm in a readable way.
static string RenderReverseState<T>(int step, Node<T>? prev, Node<T>? curr)
{
    return
$@"Step {step}
prev: {ListToArrowString(prev)}
curr: {ListToArrowString(curr)}";
}

// Runs the reverse algorithm while producing frames.
// Returns the new head.
static async Task<Node<T>?> ReverseInPlaceAnimated<T>(
    Node<T>? head,
    int delayMs = 600,
    bool tryLiveUpdate = true)
{
    Node<T>? prev = null;
    var curr = head;
    int step = 0;

    // Try to live-update (works in .NET Interactive / Polyglot in many setups).
    // If this doesn't compile in your environment, set tryLiveUpdate=false.
    if (tryLiveUpdate)
    {
        try
        {
            // "Display" and "Update" come from Microsoft.DotNet.Interactive.Formatting.
            var live = RenderReverseState(step, prev, curr).Display();

            while (curr is not null)
            {
                await Task.Delay(delayMs);

                var next = curr.Next;
                curr.Next = prev;
                prev = curr;
                curr = next;
                step++;

                live.Update(RenderReverseState(step, prev, curr));
            }

            await Task.Delay(delayMs);
            live.Update(RenderReverseState(step, prev, curr) + "\n\nDone ✅");
            return prev;
        }
        catch
        {
            // Fall through to non-live mode.
        }
    }

    // Fallback: print each frame (works everywhere).
    Console.WriteLine(RenderReverseState(step, prev, curr));
    while (curr is not null)
    {
        await Task.Delay(delayMs);

        var next = curr.Next;
        curr.Next = prev;
        prev = curr;
        curr = next;
        step++;

        Console.WriteLine();
        Console.WriteLine(RenderReverseState(step, prev, curr));
    }
    Console.WriteLine("\nDone ✅");
    return prev;
}

In [ ]:
var demo = BuildList(new[] { 1, 2, 3, 4, 5 });
PrintList(demo, "Starting list:");
demo = await ReverseInPlaceAnimated(demo, delayMs: 400, tryLiveUpdate: true);
PrintList(demo, "Final result (reversed head):");

<div dir="rtl">

## 9) בונוס: הוספה ומחיקה — פעולות נפוצות

אלו תרגילים מעולים לחיזוק עבודה עם `Next`:
- הוספה בתחילת ה<span style="color: #e91e63">רשימה</span> (PushFront)
- הוספה בסוף (Append)
- מחיקת מופע ראשון של ערך (RemoveFirst)

</div>

In [ ]:
static Node<T> PushFront<T>(Node<T>? head, T value)
{
    return new Node<T>(value, head);
}

static Node<T>? Append<T>(Node<T>? head, T value)
{
    var n = new Node<T>(value);
    if (head is null) return n;

    var cur = head;
    while (cur.Next is not null) cur = cur.Next;
    cur.Next = n;
    return head;
}

// Removes the first node whose Value equals `value`.
// Returns the (possibly new) head.
static Node<T>? RemoveFirst<T>(Node<T>? head, T value)
{
    if (head is null) return null;

    if (EqualityComparer<T>.Default.Equals(head.Value, value))
        return head.Next;

    var prev = head;
    var cur = head.Next;

    while (cur is not null)
    {
        if (EqualityComparer<T>.Default.Equals(cur.Value, value))
        {
            prev.Next = cur.Next;
            return head;
        }

        prev = cur;
        cur = cur.Next;
    }

    return head; // not found
}

In [ ]:
Node<int>? ops = null;
ops = PushFront(ops, 2);          // 2
ops = PushFront(ops, 1);          // 1 -> 2
ops = Append(ops, 3);             // 1 -> 2 -> 3
ops = Append(ops, 4);             // 1 -> 2 -> 3 -> 4
PrintList(ops, "After push/append:");

ops = RemoveFirst(ops, 1);        // remove head
PrintList(ops, "After RemoveFirst(1):");

ops = RemoveFirst(ops, 3);        // remove middle
PrintList(ops, "After RemoveFirst(3):");

ops = RemoveFirst(ops, 999);      // no-op
PrintList(ops, "After RemoveFirst(999):");

In [ ]:
#nullable enable
using System;
using Microsoft.DotNet.Interactive.Formatting;

var values = new[] { 1, 2, 3, 4, 5 };
var uid = Guid.NewGuid().ToString("N");
var valuesJs = string.Join(",", values);

var html = @"
<div dir=""rtl"" style=""font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; border:1px solid #ddd; border-radius:12px; padding:12px; max-width:1100px;"">
  <h3 style=""margin:0 0 8px 0;"">אנימציה: היפוך <span style=""color: #e91e63"">רשימה מקושרת</span> במקום (Iterative In-Place Reverse)</h3>

  <div style=""display:flex; gap:8px; flex-wrap:wrap; align-items:center; margin-bottom:10px;"">
    <button id=""play-UID"" style=""padding:6px 10px;"">נגן ▶</button>
    <button id=""pause-UID"" style=""padding:6px 10px;"">עצור ⏸</button>
    <button id=""step-UID"" style=""padding:6px 10px;"">צעד ⏭</button>
    <button id=""reset-UID"" style=""padding:6px 10px;"">איפוס ↩</button>

    <label style=""display:flex; gap:6px; align-items:center;"">
      מהירות
      <input id=""speed-UID"" type=""range"" min=""120"" max=""1400"" value=""650"" />
    </label>

    <span id=""status-UID"" style=""font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size: 13px;""></span>
  </div>

  <svg id=""svg-UID"" viewBox=""0 0 1100 280"" style=""width:100%; height:280px; background:#fbfbfd; border:1px solid #eee; border-radius:12px;""></svg>

  <div style=""margin-top:8px; color:#444; font-size:13px;"">
    בכל צעד מתבצעים בדיוק המשפטים:
    <span style=""font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace;"">
      next = curr.Next; curr.Next = prev; prev = curr; curr = next;
    </span>
  </div>
</div>

<script>
(function() {
  var uid = 'UID';
  var values = [VALUESJS];

  var next = values.map(function(_, i) { return i === values.length - 1 ? -1 : i + 1; });
  var initialNext = next.slice();

  var prev = -1;
  var curr = values.length ? 0 : -1;
  var tempNext = -1;
  var done = false;
  var phase = 'start';

  var svg = document.getElementById('svg-' + uid);
  var statusEl = document.getElementById('status-' + uid);
  var btnPlay = document.getElementById('play-' + uid);
  var btnPause = document.getElementById('pause-' + uid);
  var btnStep = document.getElementById('step-' + uid);
  var btnReset = document.getElementById('reset-' + uid);
  var speed = document.getElementById('speed-' + uid);

  var timer = null;

  function clear() { while (svg.firstChild) svg.removeChild(svg.firstChild); }

  function el(name, attrs) {
    attrs = attrs || {};
    var e = document.createElementNS('http://www.w3.org/2000/svg', name);
    for (var k in attrs) if (attrs.hasOwnProperty(k)) e.setAttribute(k, String(attrs[k]));
    return e;
  }

  function defs() {
    var d = el('defs');
    function mk(id, fill) {
      var m = el('marker', {
        id: id, viewBox: '0 0 10 10', refX: '9', refY: '5', markerWidth: '7', markerHeight: '7', orient: 'auto-start-reverse'
      });
      m.appendChild(el('path', { d: 'M 0 0 L 10 5 L 0 10 z', fill: fill }));
      d.appendChild(m);
    }
    mk('arr', '#333');
    mk('arrGreen', '#10b981');
    mk('arrBlue', '#2563eb');
    mk('arrRed', '#ef4444');
    svg.appendChild(d);
  }

  function layout(n) {
    var startX = 80, gap = 170, y = 150, w = 120, h = 54;
    var xs = [];
    for (var i = 0; i < n; i++) xs.push(startX + i * gap);
    return { xs: xs, y: y, w: w, h: h };
  }

  function nodeColor(i) {
    if (i === curr) return '#dbeafe';
    if (i === prev) return '#fee2e2';
    if (i === tempNext) return '#dcfce7';
    return 'white';
  }

  function labelColor(role) {
    if (role === 'prev') return '#ef4444';
    if (role === 'curr') return '#2563eb';
    if (role === 'next') return '#10b981';
    return '#111';
  }

  function drawNode(x, y, w, h, text, fill) {
    var g = el('g');
    g.appendChild(el('rect', { x: x, y: y, width: w, height: h, rx: 12, fill: fill, stroke: '#111', 'stroke-width': 1.6 }));
    var t = el('text', { x: x + w/2, y: y + h/2 + 6, 'text-anchor': 'middle', 'font-size': 18, fill: '#111' });
    t.textContent = text;
    g.appendChild(t);
    return g;
  }

  function drawText(x, y, text, color) {
    var t = el('text', { x: x, y: y, 'text-anchor': 'middle', 'font-size': 14, fill: color, 'font-family': 'ui-monospace, SFMono-Regular, Menlo, Consolas, monospace' });
    t.textContent = text;
    return t;
  }

  function edgePath(x1, y1, x2, y2, dir) {
    var ctrlY = dir === 'bwd' ? y1 + 55 : y1 - 55;
    return 'M ' + x1 + ' ' + y1 + ' C ' + x1 + ' ' + ctrlY + ', ' + x2 + ' ' + ctrlY + ', ' + x2 + ' ' + y2;
  }

  function drawEdge(fromX, fromY, toX, toY, dir, stroke, marker) {
    return el('path', { d: edgePath(fromX, fromY, toX, toY, dir), fill: 'none', stroke: stroke, 'stroke-width': 2.4, 'marker-end': 'url(#' + marker + ')' });
  }

  function render() {
    clear();
    defs();
    var n = values.length;
    var lay = layout(n);
    var xs = lay.xs, y = lay.y, w = lay.w, h = lay.h;

    var nullX = (n ? xs[n-1] : 80) + 190;
    var nullY = y;
    svg.appendChild(drawNode(nullX, nullY, 90, h, 'null', '#f3f4f6'));

    for (var i = 0; i < n; i++) {
      var ni = next[i];
      var fromLeft = xs[i], fromRight = xs[i] + w;
      var midY = y + h/2;

      if (ni === -1) {
        svg.appendChild(drawEdge(fromRight, midY, nullX, nullY + h/2, 'fwd', '#333', 'arr'));
      } else {
        var toLeft = xs[ni], toRight = xs[ni] + w;
        var isBackward = ni < i;
        var dir = isBackward ? 'bwd' : 'fwd';
        var stroke = isBackward ? '#a855f7' : '#333';
        svg.appendChild(drawEdge(isBackward ? fromLeft : fromRight, midY, isBackward ? toRight : toLeft, midY, dir, stroke, 'arr'));
      }
    }

    for (var i = 0; i < n; i++) {
      svg.appendChild(drawNode(xs[i], y, w, h, String(values[i]), nodeColor(i)));
    }

    var topY = 55;
    if (prev !== -1) { svg.appendChild(drawText(xs[prev] + w/2, topY, 'prev', labelColor('prev'))); }
    else { svg.appendChild(drawText(nullX + 45, topY, 'prev = null', labelColor('prev'))); }

    if (curr !== -1) { svg.appendChild(drawText(xs[curr] + w/2, topY + 20, 'curr', labelColor('curr'))); }
    else { svg.appendChild(drawText(nullX + 45, topY + 20, 'curr = null', labelColor('curr'))); }

    if (tempNext !== -1) { svg.appendChild(drawText(xs[tempNext] + w/2, topY + 40, 'next', labelColor('next'))); }
    else { svg.appendChild(drawText(nullX + 45, topY + 40, 'next = null', labelColor('next'))); }

    statusEl.textContent = 'phase=' + phase + ' | prev=' + prev + ' curr=' + curr + ' next=' + tempNext;
  }

  function oneStep() {
    if (done) return;
    if (phase === 'start') {
      tempNext = (curr === -1) ? -1 : next[curr];
      phase = 'beforeFlip';
      if (curr === -1) { done = true; phase = 'done'; }
      render();
      return;
    }
    if (phase === 'beforeFlip') {
      if (curr === -1) { done = true; phase = 'done'; render(); return; }
      next[curr] = prev;
      phase = 'afterFlip';
      render();
      return;
    }
    if (phase === 'afterFlip') {
      prev = curr;
      phase = 'afterAdvance';
      render();
      return;
    }
    if (phase === 'afterAdvance') {
      curr = tempNext;
      tempNext = (curr === -1) ? -1 : next[curr];
      if (curr === -1) { done = true; phase = 'done'; }
      else phase = 'beforeFlip';
      render();
      return;
    }
  }

  function play() {
    if (timer) return;
    timer = setInterval(function() { if (done) { pause(); return; } oneStep(); }, Number(speed.value));
  }

  function pause() { if (timer) clearInterval(timer); timer = null; }

  function reset() {
    pause();
    next = initialNext.slice();
    prev = -1;
    curr = values.length ? 0 : -1;
    tempNext = -1;
    done = false;
    phase = 'start';
    render();
  }

  btnPlay.onclick = play;
  btnPause.onclick = pause;
  btnStep.onclick = function() { oneStep(); };
  btnReset.onclick = reset;
  speed.oninput = function() { if (timer) { pause(); play(); } };

  render();
})();
</script>
".Replace("UID", uid).Replace("VALUESJS", valuesJs);

html.DisplayAs("text/html");

<div dir="rtl" style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); color: #333; padding: 20px; border-radius: 12px; margin: 30px 0;">
  <h2 style="margin: 0;">🎬 חלק ה': אנימציה ותרגילים</h2>
  <p style="margin: 10px 0 0 0; opacity: 0.8;">הדמיה ויזואלית של האלגוריתם + תרגילים לתרגול</p>
</div>

<div dir="rtl">

## תרגילים (מומלץ לפתוח תאים חדשים ולממש)

1) `FindMiddle(head)` — מציאת האמצע בעזרת “איטי/מהיר” (slow/fast).  
2) `HasCycle(head)` — זיהוי מעגל (Floyd).  
3) `ReverseKGroup(head, k)` — היפוך בקבוצות של k (מאתגר).

רוצים? הדביקו כאן את הפתרון שלכם ואני אעבור עליו איתכם.

</div>
